<a href="https://colab.research.google.com/github/ali-348/Used-Car-Price-Prediction/blob/main/Unnest_Json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [10]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
data

[{'state': 'Florida',
  'shortname': 'FL',
  'info': {'governor': 'Rick Scott'},
  'counties': [{'name': 'Dade', 'population': 12345},
   {'name': 'Broward', 'population': 40000},
   {'name': 'Palm Beach', 'population': 60000}]},
 {'state': 'Ohio',
  'shortname': 'OH',
  'info': {'governor': 'John Kasich'},
  'counties': [{'name': 'Summit', 'population': 1234},
   {'name': 'Cuyahoga', 'population': 1337}]}]

In [42]:
pd.json_normalize(data,record_path = 'counties',meta = ['state','shortname',['info','governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


# using explode

In [45]:
df=pd.DataFrame(data)
df

,state,shortname,info,counties
0,Florida,FL,{'governor': 'Rick Scott'},"[{'name': 'Dade', 'population': 12345}, {'name..."
1,Ohio,OH,{'governor': 'John Kasich'},"[{'name': 'Summit', 'population': 1234}, {'nam..."


In [46]:
new_df =df.explode('counties',ignore_index ='True') # explode to get each element in list object of counties as to new record
new_df

,state,shortname,info,counties
0,Florida,FL,{'governor': 'Rick Scott'},"{'name': 'Dade', 'population': 12345}"
1,Florida,FL,{'governor': 'Rick Scott'},"{'name': 'Broward', 'population': 40000}"
2,Florida,FL,{'governor': 'Rick Scott'},"{'name': 'Palm Beach', 'population': 60000}"
3,Ohio,OH,{'governor': 'John Kasich'},"{'name': 'Summit', 'population': 1234}"
4,Ohio,OH,{'governor': 'John Kasich'},"{'name': 'Cuyahoga', 'population': 1337}"


In [47]:
new_data = new_df.to_dict('records') # convert back to json objsct list
new_data

[{'state': 'Florida',
  'shortname': 'FL',
  'info': {'governor': 'Rick Scott'},
  'counties': {'name': 'Dade', 'population': 12345}},
 {'state': 'Florida',
  'shortname': 'FL',
  'info': {'governor': 'Rick Scott'},
  'counties': {'name': 'Broward', 'population': 40000}},
 {'state': 'Florida',
  'shortname': 'FL',
  'info': {'governor': 'Rick Scott'},
  'counties': {'name': 'Palm Beach', 'population': 60000}},
 {'state': 'Ohio',
  'shortname': 'OH',
  'info': {'governor': 'John Kasich'},
  'counties': {'name': 'Summit', 'population': 1234}},
 {'state': 'Ohio',
  'shortname': 'OH',
  'info': {'governor': 'John Kasich'},
  'counties': {'name': 'Cuyahoga', 'population': 1337}}]

In [48]:
pd.json_normalize(new_data, max_level =1)

,state,shortname,info.governor,counties.name,counties.population
0,Florida,FL,Rick Scott,Dade,12345
1,Florida,FL,Rick Scott,Broward,40000
2,Florida,FL,Rick Scott,Palm Beach,60000
3,Ohio,OH,John Kasich,Summit,1234
4,Ohio,OH,John Kasich,Cuyahoga,1337


In [7]:

pd.json_normalize(data).explode('counties')


,state,shortname,counties,info.governor
0,Florida,FL,"{'name': 'Dade', 'population': 12345}",Rick Scott
0,Florida,FL,"{'name': 'Broward', 'population': 40000}",Rick Scott
0,Florida,FL,"{'name': 'Palm Beach', 'population': 60000}",Rick Scott
1,Ohio,OH,"{'name': 'Summit', 'population': 1234}",John Kasich
1,Ohio,OH,"{'name': 'Cuyahoga', 'population': 1337}",John Kasich


In [9]:
# prompt: we can see counties is still a dictionary of key avlues pairs we need to flatten this datfarme more
df = pd.json_normalize(data).explode('counties',ignore_index='True')
df = pd.concat([df.drop(['counties'], axis=1), df['counties'].apply(pd.Series)], axis=1)
df


,state,shortname,info.governor,name,population
0,Florida,FL,Rick Scott,Dade,12345
1,Florida,FL,Rick Scott,Broward,40000
2,Florida,FL,Rick Scott,Palm Beach,60000
3,Ohio,OH,John Kasich,Summit,1234
4,Ohio,OH,John Kasich,Cuyahoga,1337


In [ ]:
# !pip install pyspark

In [11]:
# prompt: help me do the same using pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col,explode_outer
spark = SparkSession.builder.appName("JSONParsing").getOrCreate()
df = spark.createDataFrame(data)
df_exploded = df.select("state", "shortname", "info.governor", explode("counties").alias("county"))
df_flattened = df_exploded.select("state", "shortname", "governor", "county.name", "county.population")
df_flattened.show()


+-------+---------+-----------+----------+----------+
|  state|shortname|   governor|      name|population|
+-------+---------+-----------+----------+----------+
|Florida|       FL| Rick Scott|      Dade|     12345|
|Florida|       FL| Rick Scott|   Broward|     40000|
|Florida|       FL| Rick Scott|Palm Beach|     60000|
|   Ohio|       OH|John Kasich|    Summit|      1234|
|   Ohio|       OH|John Kasich|  Cuyahoga|      1337|
+-------+---------+-----------+----------+----------+



# doing it my way

In [52]:
df = spark.createDataFrame(data)
df.show()

+--------------------+--------------------+---------+-------+
|            counties|                info|shortname|  state|
+--------------------+--------------------+---------+-------+
|[{name -> Dade, p...|{governor -> Rick...|       FL|Florida|
|[{name -> Summit,...|{governor -> John...|       OH|   Ohio|
+--------------------+--------------------+---------+-------+



In [12]:
df.printSchema() # we can see counties is an array and info is an dict

root
 |-- counties: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- info: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- shortname: string (nullable = true)
 |-- state: string (nullable = true)



In [36]:
df_exploded = df.select("state", "shortname", "info.governor", explode("counties").alias("county"))
df_exploded.show() # select static columns then for the dict use a . then explode the array column

+-------+---------+-----------+--------------------+
|  state|shortname|   governor|              county|
+-------+---------+-----------+--------------------+
|Florida|       FL| Rick Scott|{name -> Dade, po...|
|Florida|       FL| Rick Scott|{name -> Broward,...|
|Florida|       FL| Rick Scott|{name -> Palm Bea...|
|   Ohio|       OH|John Kasich|{name -> Summit, ...|
|   Ohio|       OH|John Kasich|{name -> Cuyahoga...|
+-------+---------+-----------+--------------------+



In [37]:
df_exploded.printSchema() # now we can see that county is not a array anymore as we exploded it it is now a dict

root
 |-- state: string (nullable = true)
 |-- shortname: string (nullable = true)
 |-- governor: string (nullable = true)
 |-- county: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [50]:
df_exploded.select("state", "shortname", "governor", "county.name", "county.population").show()

+-------+---------+-----------+----------+----------+
|  state|shortname|   governor|      name|population|
+-------+---------+-----------+----------+----------+
|Florida|       FL| Rick Scott|      Dade|     12345|
|Florida|       FL| Rick Scott|   Broward|     40000|
|Florida|       FL| Rick Scott|Palm Beach|     60000|
|   Ohio|       OH|John Kasich|    Summit|      1234|
|   Ohio|       OH|John Kasich|  Cuyahoga|      1337|
+-------+---------+-----------+----------+----------+



In [65]:
# doing another way to get the counties as a seperste table
list_array_df = df.select(explode(col('counties')).alias(f"counties_element"))
list_array_df.show()

list_array_df.printSchema()

+--------------------+
|    counties_element|
+--------------------+
|{name -> Dade, po...|
|{name -> Broward,...|
|{name -> Palm Bea...|
|{name -> Summit, ...|
|{name -> Cuyahoga...|
+--------------------+

root
 |-- counties_element: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [67]:
# Select the struct elements inside the exploded array to flatten them
flattened_array_df = list_array_df.select('counties_element.name','counties_element.population')
flattened_array_df.show()

+----------+----------+
|      name|population|
+----------+----------+
|      Dade|     12345|
|   Broward|     40000|
|Palm Beach|     60000|
|    Summit|      1234|
|  Cuyahoga|      1337|
+----------+----------+



In [69]:
# prompt: show me another way of doing. this using pyspark but usign the with column

from pyspark.sql.functions import with_column
df_flattened = df.withColumn("county", explode("counties")) \
    .select("state", "shortname", "info.governor", "county.name", "county.population")
df_flattened.show()


+-------+---------+-----------+----------+----------+
|  state|shortname|   governor|      name|population|
+-------+---------+-----------+----------+----------+
|Florida|       FL| Rick Scott|      Dade|     12345|
|Florida|       FL| Rick Scott|   Broward|     40000|
|Florida|       FL| Rick Scott|Palm Beach|     60000|
|   Ohio|       OH|John Kasich|    Summit|      1234|
|   Ohio|       OH|John Kasich|  Cuyahoga|      1337|
+-------+---------+-----------+----------+----------+



In [73]:
# using wihtb columns
df.withColumn("county", explode("counties")).show() # creates new column called county where the arrays are explodd

+--------------------+--------------------+---------+-------+--------------------+
|            counties|                info|shortname|  state|              county|
+--------------------+--------------------+---------+-------+--------------------+
|[{name -> Dade, p...|{governor -> Rick...|       FL|Florida|{name -> Dade, po...|
|[{name -> Dade, p...|{governor -> Rick...|       FL|Florida|{name -> Broward,...|
|[{name -> Dade, p...|{governor -> Rick...|       FL|Florida|{name -> Palm Bea...|
|[{name -> Summit,...|{governor -> John...|       OH|   Ohio|{name -> Summit, ...|
|[{name -> Summit,...|{governor -> John...|       OH|   Ohio|{name -> Cuyahoga...|
+--------------------+--------------------+---------+-------+--------------------+



In [75]:
df.withColumn("county", explode("counties")).printSchema() # we can see the new column county is a map (dict) not an array

root
 |-- counties: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- info: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- shortname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [77]:
df_flattened = df.withColumn("county", explode("counties")) \
    .select("state", "shortname", "info.governor", "county.name", "county.population").show()

+-------+---------+-----------+----------+----------+
|  state|shortname|   governor|      name|population|
+-------+---------+-----------+----------+----------+
|Florida|       FL| Rick Scott|      Dade|     12345|
|Florida|       FL| Rick Scott|   Broward|     40000|
|Florida|       FL| Rick Scott|Palm Beach|     60000|
|   Ohio|       OH|John Kasich|    Summit|      1234|
|   Ohio|       OH|John Kasich|  Cuyahoga|      1337|
+-------+---------+-----------+----------+----------+



In [82]:
df_flattened = df.withColumn("county", explode("counties")) \
    .select("county.name", "county.population").show()

+----------+----------+
|      name|population|
+----------+----------+
|      Dade|     12345|
|   Broward|     40000|
|Palm Beach|     60000|
|    Summit|      1234|
|  Cuyahoga|      1337|
+----------+----------+



# More complex data

In [12]:
data = {
  "orders": [
    {
      "order_id": 12345,
      "order_date": "2024-08-02T12:34:56Z",
      "customer": {
        "id": 9876,
        "name": "John Doe",
        "email": "johndoe@example.com",
        "address": {
          "street": "123 Main St",
          "city": "Anytown",
          "state": "CA",
          "zip": "12345"
        }
      },
      "shipping_address": {
        "name": "Jane Doe",
        "street": "456 Elm St",
        "city": "Othertown",
        "state": "NY",
        "zip": "67890"
      },
      "billing_address": {
        "street": "123 Main St",
        "city": "Anytown",
        "state": "CA",
        "zip": "12345"
      },
      "products": [
        {
          "product_id": 101,
          "name": "Laptop",
          "price": 999.99,
          "quantity": 1,
          "options": {
            "color": "silver",
            "storage": "512GB"
          }
        },
        {
          "product_id": 202,
          "name": "Headphones",
          "price": 199.99,
          "quantity": 2
        }
      ],
      "total_price": 1199.97,
      "shipping_method": "standard",
      "payment_method": "credit_card",
      "status": "shipped"
    }
  ]
}
data

{'orders': [{'order_id': 12345,
   'order_date': '2024-08-02T12:34:56Z',
   'customer': {'id': 9876,
    'name': 'John Doe',
    'email': 'johndoe@example.com',
    'address': {'street': '123 Main St',
     'city': 'Anytown',
     'state': 'CA',
     'zip': '12345'}},
   'shipping_address': {'name': 'Jane Doe',
    'street': '456 Elm St',
    'city': 'Othertown',
    'state': 'NY',
    'zip': '67890'},
   'billing_address': {'street': '123 Main St',
    'city': 'Anytown',
    'state': 'CA',
    'zip': '12345'},
   'products': [{'product_id': 101,
     'name': 'Laptop',
     'price': 999.99,
     'quantity': 1,
     'options': {'color': 'silver', 'storage': '512GB'}},
    {'product_id': 202, 'name': 'Headphones', 'price': 199.99, 'quantity': 2}],
   'total_price': 1199.97,
   'shipping_method': 'standard',
   'payment_method': 'credit_card',
   'status': 'shipped'}]}

In [13]:
df = spark.createDataFrame(data['orders'])
df.show()

+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+---------------+-------+-----------+
|     billing_address|            customer|          order_date|order_id|payment_method|            products|    shipping_address|shipping_method| status|total_price|
+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+---------------+-------+-----------+
|{zip -> 12345, st...|{name -> NULL, ad...|2024-08-02T12:34:56Z|   12345|   credit_card|[{options -> NULL...|{zip -> 67890, na...|       standard|shipped|    1199.97|
+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+---------------+-------+-----------+



In [14]:
df.printSchema() # we can se products is an array type

root
 |-- billing_address: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- customer: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- shipping_address: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- shipping_method: string (nullable = true)
 |-- status: string (nullable = true)
 |-- total_price: double (nullable = true)



In [18]:
array_column_name = 'products'
products_exploded_df = df.withColumn('products_exploded',explode(array_column_name)).select('*') #
products_exploded_df.show()# this code creates a new column that is the products column exploded then we select all the original column as well

+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+---------------+-------+-----------+--------------------+
|     billing_address|            customer|          order_date|order_id|payment_method|            products|    shipping_address|shipping_method| status|total_price|   products_exploded|
+--------------------+--------------------+--------------------+--------+--------------+--------------------+--------------------+---------------+-------+-----------+--------------------+
|{zip -> 12345, st...|{name -> NULL, ad...|2024-08-02T12:34:56Z|   12345|   credit_card|[{options -> NULL...|{zip -> 67890, na...|       standard|shipped|    1199.97|{options -> NULL,...|
|{zip -> 12345, st...|{name -> NULL, ad...|2024-08-02T12:34:56Z|   12345|   credit_card|[{options -> NULL...|{zip -> 67890, na...|       standard|shipped|    1199.97|{name -> NULL, qu...|
+--------------------+--------------------+-----------------

In [20]:
products_exploded_df.printSchema() # we can see products exploded is no longer an array type

root
 |-- billing_address: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- customer: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- shipping_address: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- shipping_method: string (nullable = true)
 |-- status: string (nullable = true)
 |-- total_price: double (nullable = true)
 |-- products_exploded: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)



In [39]:
# if we only wanted the products exploded we do this
array_column_name = 'products'
products_exploded_df = df.withColumn('products_exploded',explode(array_column_name)).select('products_exploded')
products_exploded_df.show()
products_exploded_df.printSchema()

+--------------------+
|   products_exploded|
+--------------------+
|{options -> NULL,...|
|{name -> NULL, qu...|
+--------------------+

root
 |-- products_exploded: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)



In [38]:
other_way = df.select(explode(col('products')).alias(f"products_exploded"))
other_way.show()
other_way.printSchema()

+--------------------+
|   products_exploded|
+--------------------+
|{options -> NULL,...|
|{name -> NULL, qu...|
+--------------------+

root
 |-- products_exploded: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)



In [40]:
products_exploded_df.select('products_exploded.product_id','products_exploded.name','products_exploded.price','products_exploded.quantity').show()

+----------+----+-----+--------+
|product_id|name|price|quantity|
+----------+----+-----+--------+
|       101|NULL| NULL|       1|
|       202|NULL| NULL|       2|
+----------+----+-----+--------+



# New dataset

In [45]:
import requests
import json
response = requests.get('https://dummyjson.com/carts')


In [76]:
raw_data = response.json()
raw_data

{'carts': [{'id': 1,
   'products': [{'id': 168,
     'title': 'Charger SXT RWD',
     'price': 32999.99,
     'quantity': 3,
     'total': 98999.97,
     'discountPercentage': 13.39,
     'discountedTotal': 85743.87,
     'thumbnail': 'https://cdn.dummyjson.com/products/images/vehicle/Charger%20SXT%20RWD/thumbnail.png'},
    {'id': 78,
     'title': 'Apple MacBook Pro 14 Inch Space Grey',
     'price': 1999.99,
     'quantity': 2,
     'total': 3999.98,
     'discountPercentage': 18.52,
     'discountedTotal': 3259.18,
     'thumbnail': 'https://cdn.dummyjson.com/products/images/laptops/Apple%20MacBook%20Pro%2014%20Inch%20Space%20Grey/thumbnail.png'},
    {'id': 183,
     'title': 'Green Oval Earring',
     'price': 24.99,
     'quantity': 5,
     'total': 124.94999999999999,
     'discountPercentage': 6.28,
     'discountedTotal': 117.1,
     'thumbnail': 'https://cdn.dummyjson.com/products/images/womens-jewellery/Green%20Oval%20Earring/thumbnail.png'},
    {'id': 100,
     'title': 

In [48]:
type(raw_data) # this is a dict we wan t jst the carts data

dict

In [77]:
data = raw_data['carts']
print(type(data))
print(len(data))
# data = raw_data.get('carts') # same thing

<class 'list'>
30


In [59]:
#eading data into pandas dataframe

df = pd.DataFrame(data)
df.head() # we can see products is an json list object

,id,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,"[{'id': 168, 'title': 'Charger SXT RWD', 'pric...",103774.85,89686.65,33,4,15
1,2,"[{'id': 144, 'title': 'Cricket Helmet', 'price...",4794.80,4288.95,142,5,20
2,3,"[{'id': 98, 'title': 'Rolex Submariner Watch',...",16775.87,14144.30,108,5,13
3,4,"[{'id': 187, 'title': 'Golden Shoes Woman', 'p...",456.83,443.23,87,4,17
4,5,"[{'id': 108, 'title': 'iPhone 12 Silicone Case...",7431.30,6676.44,134,6,20


In [70]:
pd.json_normalize(data,record_path = 'products',meta = ['id','total','discountedTotal','userId','totalProducts','totalQuantity'],record_prefix='products.' )

,products.id,products.title,products.price,products.quantity,products.total,products.discountPercentage,products.discountedTotal,products.thumbnail,id,total,discountedTotal,userId,totalProducts,totalQuantity
0,168,Charger SXT RWD,32999.99,3,98999.97,13.39,85743.87,https://cdn.dummyjson.com/products/images/vehi...,1,103774.85,89686.65,33,4,15
1,78,Apple MacBook Pro 14 Inch Space Grey,1999.99,2,3999.98,18.52,3259.18,https://cdn.dummyjson.com/products/images/lapt...,1,103774.85,89686.65,33,4,15
2,183,Green Oval Earring,24.99,5,124.95,6.28,117.10,https://cdn.dummyjson.com/products/images/wome...,1,103774.85,89686.65,33,4,15
3,100,Apple Airpods,129.99,5,649.95,12.84,566.50,https://cdn.dummyjson.com/products/images/mobi...,1,103774.85,89686.65,33,4,15
4,144,Cricket Helmet,44.99,4,179.96,11.47,159.32,https://cdn.dummyjson.com/products/images/spor...,2,4794.8,4288.95,142,5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,153,Volleyball,11.99,5,59.95,16.05,50.33,https://cdn.dummyjson.com/products/images/spor...,29,3862.43,3488.44,170,5,17
117,181,Marni Red & Black Suit,179.99,1,179.99,14.21,154.41,https://cdn.dummyjson.com/products/images/wome...,30,128249.07,118740.76,177,4,13
118,171,Pacifica Touring,31999.99,4,127999.96,7.40,118527.96,https://cdn.dummyjson.com/products/images/vehi...,30,128249.07,118740.76,177,4,13
119,35,Potatoes,2.29,4,9.16,1.69,9.01,https://cdn.dummyjson.com/products/images/groc...,30,128249.07,118740.76,177,4,13


In [64]:
 # using pandas explode and normalise together
new_df = df.explode('products',ignore_index ='True')
new_df # this will get each item in list as its own record


,id,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,"{'id': 168, 'title': 'Charger SXT RWD', 'price...",103774.85,89686.65,33,4,15
1,1,"{'id': 78, 'title': 'Apple MacBook Pro 14 Inch...",103774.85,89686.65,33,4,15
2,1,"{'id': 183, 'title': 'Green Oval Earring', 'pr...",103774.85,89686.65,33,4,15
3,1,"{'id': 100, 'title': 'Apple Airpods', 'price':...",103774.85,89686.65,33,4,15
4,2,"{'id': 144, 'title': 'Cricket Helmet', 'price'...",4794.80,4288.95,142,5,20
...,...,...,...,...,...,...,...
116,29,"{'id': 153, 'title': 'Volleyball', 'price': 11...",3862.43,3488.44,170,5,17
117,30,"{'id': 181, 'title': 'Marni Red & Black Suit',...",128249.07,118740.76,177,4,13
118,30,"{'id': 171, 'title': 'Pacifica Touring', 'pric...",128249.07,118740.76,177,4,13
119,30,"{'id': 35, 'title': 'Potatoes', 'price': 2.29,...",128249.07,118740.76,177,4,13


In [71]:
new_data = new_df.to_dict('records')
pd.json_normalize(new_data, max_level =1)

,id,total,discountedTotal,userId,totalProducts,totalQuantity,products.id,products.title,products.price,products.quantity,products.total,products.discountPercentage,products.discountedTotal,products.thumbnail
0,1,103774.85,89686.65,33,4,15,168,Charger SXT RWD,32999.99,3,98999.97,13.39,85743.87,https://cdn.dummyjson.com/products/images/vehi...
1,1,103774.85,89686.65,33,4,15,78,Apple MacBook Pro 14 Inch Space Grey,1999.99,2,3999.98,18.52,3259.18,https://cdn.dummyjson.com/products/images/lapt...
2,1,103774.85,89686.65,33,4,15,183,Green Oval Earring,24.99,5,124.95,6.28,117.10,https://cdn.dummyjson.com/products/images/wome...
3,1,103774.85,89686.65,33,4,15,100,Apple Airpods,129.99,5,649.95,12.84,566.50,https://cdn.dummyjson.com/products/images/mobi...
4,2,4794.80,4288.95,142,5,20,144,Cricket Helmet,44.99,4,179.96,11.47,159.32,https://cdn.dummyjson.com/products/images/spor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,29,3862.43,3488.44,170,5,17,153,Volleyball,11.99,5,59.95,16.05,50.33,https://cdn.dummyjson.com/products/images/spor...
117,30,128249.07,118740.76,177,4,13,181,Marni Red & Black Suit,179.99,1,179.99,14.21,154.41,https://cdn.dummyjson.com/products/images/wome...
118,30,128249.07,118740.76,177,4,13,171,Pacifica Touring,31999.99,4,127999.96,7.40,118527.96,https://cdn.dummyjson.com/products/images/vehi...
119,30,128249.07,118740.76,177,4,13,35,Potatoes,2.29,4,9.16,1.69,9.01,https://cdn.dummyjson.com/products/images/groc...


In [80]:
## using spark
df = spark.createDataFrame(data)
df.show()

+---------------+---+--------------------+---------+-------------+-------------+------+
|discountedTotal| id|            products|    total|totalProducts|totalQuantity|userId|
+---------------+---+--------------------+---------+-------------+-------------+------+
|       89686.65|  1|[{discountedTotal...|103774.85|            4|           15|    33|
|        4288.95|  2|[{discountedTotal...|   4794.8|            5|           20|   142|
|        14144.3|  3|[{discountedTotal...| 16775.87|            5|           13|   108|
|         443.23|  4|[{discountedTotal...|   456.83|            4|           17|    87|
|        6676.44|  5|[{discountedTotal...|   7431.3|            6|           20|   134|
|       31435.05|  6|[{discountedTotal...| 35199.86|            4|           14|   150|
|      128504.41|  7|[{discountedTotal...|145651.89|            3|           11|    86|
|       12666.75|  8|[{discountedTotal...| 15685.86|            4|           14|    23|
|       63336.57|  9|[{discounte

In [81]:
df.printSchema() # we can see products is an array type

root
 |-- discountedTotal: double (nullable = true)
 |-- id: long (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- total: double (nullable = true)
 |-- totalProducts: long (nullable = true)
 |-- totalQuantity: long (nullable = true)
 |-- userId: long (nullable = true)



In [101]:
products_exploded = df.withColumn('products_exploded',explode_outer('products')).select('products_exploded')
products_exploded.show()

+--------------------+
|   products_exploded|
+--------------------+
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
|{discountedTotal ...|
+--------------------+
only showing top 20 rows



In [107]:
products_exploded.select('products_exploded.id','products_exploded.title').show()

+---+-----+
| id|title|
+---+-----+
|168| NULL|
| 78| NULL|
|183| NULL|
|100| NULL|
|144| NULL|
|124| NULL|
|148| NULL|
|122| NULL|
|110| NULL|
| 98| NULL|
|125| NULL|
| 55| NULL|
| 62| NULL|
|132| NULL|
|187| NULL|
| 40| NULL|
|156| NULL|
| 62| NULL|
|108| NULL|
|138| NULL|
+---+-----+
only showing top 20 rows



# new c_data

In [108]:
raw_data = {
    "c_DATA": [
      {
        "guid": "1a2b3c4d5e",
        "name": "John Doe",
        "age": "30",
        "c_action": {
          "guid": "a1b2c3d4e5",
          "action_type": "click",
          "timestamp": "2023-06-01T12:34:56Z"
        },
        "emails": {
          "guid": "f1g2h3i4j5",
          "email_address": "john.doe@example.com",
          "verified": "true"
        },
        "s_posts": {
          "guid": "k1l2m3n4o5",
          "post_content": "Hello World",
          "post_date": "2023-06-01"
        },
        "land_page": {
          "guid": "p1q2r3s4t5",
          "page_url": "http://example.com",
          "visit_duration": "5 minutes"
        },
        "form": {
          "guid": "u1v2w3x4y5",
          "form_name": "Contact Us",
          "submission_date": "2023-06-01"
        },
        "trsource": {
          "guid": "z1a2b3c4d5",
          "source_type": "referral",
          "source_detail": "google.com"
        }
      },
      {
        "guid": "6f7g8h9i0j",
        "name": "Jane Smith",
        "age": "25",
        "c_action": {
          "guid": "b1c2d3e4f5",
          "action_type": "view",
          "timestamp": "2023-06-02T14:22:33Z"
        },
        "emails": {
          "guid": "g1h2i3j4k5",
          "email_address": "jane.smith@example.com",
          "verified": "false"
        },
        "s_posts": {
          "guid": "l1m2n3o4p5",
          "post_content": "Goodbye World",
          "post_date": "2023-06-02"
        },
        "land_page": {
          "guid": "q1r2s3t4u5",
          "page_url": "http://example.org",
          "visit_duration": "10 minutes"
        },
        "form": {
          "guid": "v1w2x3y4z5",
          "form_name": "Sign Up",
          "submission_date": "2023-06-02"
        },
        "trsource": {
          "guid": "a2b3c4d5e6",
          "source_type": "direct",
          "source_detail": "example.com"
        }
      }
    ],
    "other_array1": [
      {
        "field1": "value1",
        "field2": "value2"
      },
      {
        "field1": "value3",
        "field2": "value4"
      }
    ],
    "other_array2": [
      {
        "field1": "value5",
        "field2": "value6"
      },
      {
        "field1": "value7",
        "field2": "value8"
      }
    ],
    "other_array3": [
      {
        "field1": "value9",
        "field2": "value10"
      },
      {
        "field1": "value11",
        "field2": "value12"
      }
    ]
  }


In [109]:
data = raw_data['c_DATA']
print(type(data))
print(len(data))

<class 'list'>
2


In [112]:
df = pd.DataFrame(data)
df # so we can see inside of c_data there are no actual list of json objects iside just regualr lists

,guid,name,age,c_action,emails,s_posts,land_page,form,trsource
0,1a2b3c4d5e,John Doe,30,"{'guid': 'a1b2c3d4e5', 'action_type': 'click',...","{'guid': 'f1g2h3i4j5', 'email_address': 'john....","{'guid': 'k1l2m3n4o5', 'post_content': 'Hello ...","{'guid': 'p1q2r3s4t5', 'page_url': 'http://exa...","{'guid': 'u1v2w3x4y5', 'form_name': 'Contact U...","{'guid': 'z1a2b3c4d5', 'source_type': 'referra..."
1,6f7g8h9i0j,Jane Smith,25,"{'guid': 'b1c2d3e4f5', 'action_type': 'view', ...","{'guid': 'g1h2i3j4k5', 'email_address': 'jane....","{'guid': 'l1m2n3o4p5', 'post_content': 'Goodby...","{'guid': 'q1r2s3t4u5', 'page_url': 'http://exa...","{'guid': 'v1w2x3y4z5', 'form_name': 'Sign Up',...","{'guid': 'a2b3c4d5e6', 'source_type': 'direct'..."


In [126]:
pd.json_normalize(data) # this will flatten out everything into one table

,guid,name,age,c_action.guid,c_action.action_type,c_action.timestamp,emails.guid,emails.email_address,emails.verified,s_posts.guid,...,s_posts.post_date,land_page.guid,land_page.page_url,land_page.visit_duration,form.guid,form.form_name,form.submission_date,trsource.guid,trsource.source_type,trsource.source_detail
0,1a2b3c4d5e,John Doe,30,a1b2c3d4e5,click,2023-06-01T12:34:56Z,f1g2h3i4j5,john.doe@example.com,true,k1l2m3n4o5,...,2023-06-01,p1q2r3s4t5,http://example.com,5 minutes,u1v2w3x4y5,Contact Us,2023-06-01,z1a2b3c4d5,referral,google.com
1,6f7g8h9i0j,Jane Smith,25,b1c2d3e4f5,view,2023-06-02T14:22:33Z,g1h2i3j4k5,jane.smith@example.com,false,l1m2n3o4p5,...,2023-06-02,q1r2s3t4u5,http://example.org,10 minutes,v1w2x3y4z5,Sign Up,2023-06-02,a2b3c4d5e6,direct,example.com


In [142]:
# prompt: how would i sue json normalise ifni just wanted soeting like the c_action as its own table
array_column = 'c_action'
static_column = ['guid','name','age']
# Combine the columns
columns = [array_column] + static_column

# get c_action as its own dataframe
c_action = df[columns]
c_action

,c_action,guid,name,age
0,"{'guid': 'a1b2c3d4e5', 'action_type': 'click',...",1a2b3c4d5e,John Doe,30
1,"{'guid': 'b1c2d3e4f5', 'action_type': 'view', ...",6f7g8h9i0j,Jane Smith,25


In [141]:
c_action_data = c_action.to_dict('records')
c_action_flattened  = pd.json_normalize(c_action_data)
c_action_flattened

,guid,name,age,c_action.guid,c_action.action_type,c_action.timestamp
0,1a2b3c4d5e,John Doe,30,a1b2c3d4e5,click,2023-06-01T12:34:56Z
1,6f7g8h9i0j,Jane Smith,25,b1c2d3e4f5,view,2023-06-02T14:22:33Z


In [143]:
c_action_flattened.to_csv('c_action.csv')

In [144]:
# making a function that does it in a loop
def flatten_and_write_each_array(data, array_column_name):
  # read json data into dataframe
  df = pd.DataFrame(data)
  array_column = array_column_name
  # get static columns
  static_column = ['guid','name','age']
  # Combine the columns
  columns = [array_column] + static_column
  # get each array column as its own dataframe with the static columns
  df_array_column = df[columns]
  # convert back to json
  array_column_data = df_array_column.to_dict('records')
  # flatten the array column
  df_array_column_flattened  = pd.json_normalize(array_column_data)
  # write to csv
  df_array_column_flattened.to_csv(f'{array_column_name}.csv')

  return df_array_column_flattened




In [146]:
flatten_and_write_each_array(data,'emails')

In [147]:
# run it in a for each loop
array_columns = ['emails','s_posts','land_page','form','trsource','c_action']
for array_column in array_columns:
  flatten_and_write_each_array(data,array_column)

In [148]:
spark_df = spark.createDataFrame(data)
spark_df.show()
spark_df.printSchema()

+---+--------------------+--------------------+--------------------+----------+--------------------+----------+--------------------+--------------------+
|age|            c_action|              emails|                form|      guid|           land_page|      name|             s_posts|            trsource|
+---+--------------------+--------------------+--------------------+----------+--------------------+----------+--------------------+--------------------+
| 30|{guid -> a1b2c3d4...|{verified -> true...|{submission_date ...|1a2b3c4d5e|{page_url -> http...|  John Doe|{guid -> k1l2m3n4...|{source_detail ->...|
| 25|{guid -> b1c2d3e4...|{verified -> fals...|{submission_date ...|6f7g8h9i0j|{page_url -> http...|Jane Smith|{guid -> l1m2n3o4...|{source_detail ->...|
+---+--------------------+--------------------+--------------------+----------+--------------------+----------+--------------------+--------------------+

root
 |-- age: string (nullable = true)
 |-- c_action: map (nullable = true

In [152]:
array_column_name = 'c_action'
c_data_explode_df = spark_df.select(explode(col(array_column_name)))
c_data_explode_df.show()

+-----------+--------------------+
|        key|               value|
+-----------+--------------------+
|       guid|          a1b2c3d4e5|
|action_type|               click|
|  timestamp|2023-06-01T12:34:56Z|
|       guid|          b1c2d3e4f5|
|action_type|                view|
|  timestamp|2023-06-02T14:22:33Z|
+-----------+--------------------+



In [157]:
# using slicer indexing
data[0]['c_action']
data[1]['c_action']

{'guid': 'b1c2d3e4f5',
 'action_type': 'view',
 'timestamp': '2023-06-02T14:22:33Z'}

In [158]:
# prompt: write me code that ususes json slicer indexing so that i cna get each of the arrays for a specific key in a datafrem e.g. data[0]['c_action']
# data[1]['c_action'] both if these should be in one table cakled c_action and so on

import pandas as pd

def extract_nested_data(data, key):
  """
  Extracts nested data for a specific key from a list of dictionaries.

  Args:
    data: A list of dictionaries.
    key: The key for which to extract nested data.

  Returns:
    A pandas DataFrame containing the extracted data.
  """

  extracted_data = []
  for item in data:
    if key in item:
      extracted_data.append(item[key])

  return pd.DataFrame(extracted_data)

# Example usage:
c_action_df = extract_nested_data(data, 'c_action')
print(c_action_df)

emails_df = extract_nested_data(data, 'emails')
print(emails_df)


         guid action_type             timestamp
0  a1b2c3d4e5       click  2023-06-01T12:34:56Z
1  b1c2d3e4f5        view  2023-06-02T14:22:33Z
         guid           email_address verified
0  f1g2h3i4j5    john.doe@example.com     true
1  g1h2i3j4k5  jane.smith@example.com    false


In [183]:
import pandas as pd

def extract_nested_data(data, key):
  """
  Extracts nested data for a specific key from a list of dictionaries, handling various data structures.
  Includes static values from the original data in the extracted DataFrame.

  Args:
    data: A list of dictionaries or a complex nested data structure.
    key: The key for which to extract nested data.

  Returns:
    A pandas DataFrame containing the extracted data and static values.
  """

  def extract_helper(item, key):
    try:
      if isinstance(item, dict):
        if key in item:
          return item[key]
        else:
          return {k: v for k, v in item.items() if not isinstance(v, dict) and not isinstance(v, list)}
      elif isinstance(item, list):
        results = []
        for sub_item in item:
          result = extract_helper(sub_item, key)
          if result is not None:
            results.append(result)
        return results if results else None
      else:
        return None
    except Exception as e:
      print(f"Error extracting data: {e}")
      return None

  extracted_data = []
  for item in data:
    try:
      result = extract_helper(item, key)
      if result is not None:
        extracted_data.append(result)
    except Exception as e:
      print(f"Error processing item: {e}")

  if extracted_data:
    df = pd.DataFrame(extracted_data)
    # Handle potential empty DataFrames
    if df.empty:
      return None
    return df
  else:
    return None


In [184]:
extract_nested_data(data,'form')

,guid,form_name,submission_date
0,u1v2w3x4y5,Contact Us,2023-06-01
1,v1w2x3y4z5,Sign Up,2023-06-02


In [219]:
key_name = 'emails'

extracted_data = []
for item in data:
  if key_name in item:
    array = item[key_name]
    extracted_data.append(array)
extracted_data


[{'guid': 'f1g2h3i4j5',
  'email_address': 'john.doe@example.com',
  'verified': 'true'},
 {'guid': 'g1h2i3j4k5',
  'email_address': 'jane.smith@example.com',
  'verified': 'false'}]

In [220]:
pd.DataFrame(extracted_data)

,guid,email_address,verified
0,f1g2h3i4j5,john.doe@example.com,true
1,g1h2i3j4k5,jane.smith@example.com,false


In [218]:
extracted_data

[]